# Navier's Solution to Simply-Supported Kirchhoff-Love Plates


The notebook goes through the steps of symbolicly getting the expressions that are turned to code in the library. The steps lack explanation at the moment, later I'll come back to it (the author).

In [78]:
from sympy import symbols, Function, diff, Matrix, integrate, sin, cos, pi, simplify

## Displacement Field


In [79]:
# coordinates in a suitable vector space
x, y, z = symbols("x y z", real=True)

# kinematic variables
w0 = Function("w_0")(x, y)
thx = Function("\\theta_x")(x, y)
thy = Function("\\theta_y")(x, y)

u = z * thy
v = -z * thx
w = w0
disps = [u, v, w]
Matrix(disps)

Matrix([
[ z*\theta_y(x, y)],
[-z*\theta_x(x, y)],
[        w_0(x, y)]])

## Boundary Conditions and Trial Solution


In [80]:
mn = m, n = symbols("m n", integer=True, positive=True)
coeff = Amn = symbols("A_{mn}", real=True)
shape = Lx, Ly = symbols("L_x, L_y", real=True)
Sm, Sn = sin(m * pi * x / Lx), sin(n * pi * y / Ly)
Cm, Cn = cos(m * pi * x / Lx), cos(n * pi * y / Ly)

In [81]:
w0_trial = Amn * Sm * Sn
thx_trial = diff(w0_trial, y)
thy_trial = -diff(w0_trial, x)
trial = {w0: w0_trial, thx: thx_trial, thy: thy_trial}
disps_trial = [s.subs(trial).expand().doit() for s in [w0_trial, thx_trial, thy_trial]]
Matrix(disps_trial)

Matrix([
[          A_{mn}*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)],
[ pi*A_{mn}*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y],
[-pi*A_{mn}*m*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x]])

In [82]:
coeff_load = pmn, qmn, rmn = symbols("p_{mn} q_{mn} r_{mn}", real=True)
pz_trial = pmn * Sm * Sn
pxx_trial = qmn * Sm * Cn
pyy_trial = rmn * Cm * Sn
Matrix([pz_trial, pxx_trial, pyy_trial])

Matrix([
[p_{mn}*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)],
[q_{mn}*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)],
[r_{mn}*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)]])

## Strain - Displacement Equations


In [83]:
exx = diff(u, x)
eyy = diff(v, y)
exy = diff(u, y) + diff(v, x)
strains = [exx, eyy, exy]
Matrix(strains)

Matrix([
[                                   z*Derivative(\theta_y(x, y), x)],
[                                  -z*Derivative(\theta_x(x, y), y)],
[-z*Derivative(\theta_x(x, y), x) + z*Derivative(\theta_y(x, y), y)]])

In [84]:
kxx = diff(exx, z)
kyy = diff(eyy, z)
kxy = diff(exy, z)
gen_strains = [kxx, kyy, kxy]
Matrix(gen_strains)

Matrix([
[                                 Derivative(\theta_y(x, y), x)],
[                                -Derivative(\theta_x(x, y), y)],
[-Derivative(\theta_x(x, y), x) + Derivative(\theta_y(x, y), y)]])

In [85]:
gen_strains_trial = [k.subs(trial).expand().doit() for k in gen_strains]
Matrix(gen_strains_trial)

Matrix([
[     pi**2*A_{mn}*m**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x**2],
[     pi**2*A_{mn}*n**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y**2],
[-2*pi**2*A_{mn}*m*n*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/(L_x*L_y)]])

## Stress Resultants


In [86]:
D11, D22, D12, D66 = Dij = symbols("D_11 D_22 D_12 D_66", real=True)
#
kx, ky, kxy = gen_strains
#
mx = D11 * kx + D12 * ky
my = D12 * kx + D22 * ky
mxy = D66 * kxy
#
mx = mx.simplify().expand()
my = my.simplify().expand()
mxy = mxy.simplify().expand()

vx = diff(mx, x) + diff(mxy, y) + pyy_trial
vy = diff(mxy, x) + diff(my, y) - pxx_trial

gen_forces = [mx, my, mxy]
Matrix(gen_forces)

Matrix([
[ D_11*Derivative(\theta_y(x, y), x) - D_12*Derivative(\theta_x(x, y), y)],
[ D_12*Derivative(\theta_y(x, y), x) - D_22*Derivative(\theta_x(x, y), y)],
[-D_66*Derivative(\theta_x(x, y), x) + D_66*Derivative(\theta_y(x, y), y)]])

In [87]:
gen_forces_trial = [k.subs(trial).expand().doit() for k in gen_forces]
Matrix(gen_forces_trial)

Matrix([
[pi**2*A_{mn}*D_11*m**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x**2 + pi**2*A_{mn}*D_12*n**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y**2],
[pi**2*A_{mn}*D_12*m**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x**2 + pi**2*A_{mn}*D_22*n**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y**2],
[                                                           -2*pi**2*A_{mn}*D_66*m*n*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/(L_x*L_y)]])

## Equilibrium Equations


The signs of the equations
is selected such, that the coefficients of the load functions on the
right-hand sides admit positive signs according to global axes.


In [88]:
# vertical equilibrium
lhs_fz = -simplify(diff(mx, x, 2) + diff(my, y, 2) + 2 * diff(mxy, x, y)).expand()
lhs_fz = lhs_fz.expand().doit() / (Sn * Sm)
lhs_fz = lhs_fz.expand().simplify()

equations = [lhs_fz]
Matrix(equations)

Matrix([[(-D_11*Derivative(\theta_y(x, y), (x, 3)) + D_12*Derivative(\theta_x(x, y), (x, 2), y) - D_12*Derivative(\theta_y(x, y), x, (y, 2)) + D_22*Derivative(\theta_x(x, y), (y, 3)) + 2*D_66*Derivative(\theta_x(x, y), (x, 2), y) - 2*D_66*Derivative(\theta_y(x, y), x, (y, 2)))/(sin(pi*m*x/L_x)*sin(pi*n*y/L_y))]])

In [89]:
equations_trial = [k.subs(trial).expand().doit() for k in equations]
Matrix(equations_trial)

Matrix([[pi**4*A_{mn}*D_11*m**4/L_x**4 + 2*pi**4*A_{mn}*D_12*m**2*n**2/(L_x**2*L_y**2) + pi**4*A_{mn}*D_22*n**4/L_y**4 + 4*pi**4*A_{mn}*D_66*m**2*n**2/(L_x**2*L_y**2)]])

## Loads


### Constant


#### Constans vertical load over a rectangular area


In [90]:
# constans vertical load over a rectangular area
q, w, h = symbols("q w h", real=True)
xc, yc = symbols("x_c y_c", real=True)
qmn = (4 / (Lx * Ly)) * integrate(
    q * Sm * Sn, (x, xc - w / 2, xc + w / 2), (y, yc - h / 2, yc + h / 2)
)
qmn_fz = qmn.simplify().expand()
qmn_fz

16*q*sin(pi*m*w/(2*L_x))*sin(pi*m*x_c/L_x)*sin(pi*h*n/(2*L_y))*sin(pi*n*y_c/L_y)/(pi**2*m*n)

#### Constans $m_x$ moment over a rectangular area


In [91]:
# constans moment of intensity mx, around global axis X,
# over a rectangular area with width w, height h and center (xc, yc)
m_x, w, h = symbols("m_x w h", real=True)
xc, yc = symbols("x_c y_c", real=True)
qmn = (4 / (Lx * Ly)) * integrate(
    m_x * Sm * Cn, (x, xc - w / 2, xc + w / 2), (y, yc - h / 2, yc + h / 2)
)
qmn_mx = qmn.simplify().expand()
qmn_mx

16*m_x*sin(pi*m*w/(2*L_x))*sin(pi*m*x_c/L_x)*sin(pi*h*n/(2*L_y))*cos(pi*n*y_c/L_y)/(pi**2*m*n)

#### Constans $m_y$ moment over a rectangular area


In [92]:
# constant moment of intensity my, around global axis Y,
# over a rectangular area with width w, height h and center (xc, yc)
m_y, w, h = symbols("m_y w h", real=True)
xc, yc = symbols("x_c y_c", real=True)
qmn = (4 / (Lx * Ly)) * integrate(
    m_y * Cm * Sn, (x, xc - w / 2, xc + w / 2), (y, yc - h / 2, yc + h / 2)
)
qmn_my = qmn.simplify().expand()
qmn_my

16*m_y*sin(pi*m*w/(2*L_x))*sin(pi*h*n/(2*L_y))*sin(pi*n*y_c/L_y)*cos(pi*m*x_c/L_x)/(pi**2*m*n)

In [93]:
f = Matrix([qmn_fz, qmn_mx, qmn_my])
f

Matrix([
[  16*q*sin(pi*m*w/(2*L_x))*sin(pi*m*x_c/L_x)*sin(pi*h*n/(2*L_y))*sin(pi*n*y_c/L_y)/(pi**2*m*n)],
[16*m_x*sin(pi*m*w/(2*L_x))*sin(pi*m*x_c/L_x)*sin(pi*h*n/(2*L_y))*cos(pi*n*y_c/L_y)/(pi**2*m*n)],
[16*m_y*sin(pi*m*w/(2*L_x))*sin(pi*h*n/(2*L_y))*sin(pi*n*y_c/L_y)*cos(pi*m*x_c/L_x)/(pi**2*m*n)]])

### Postprocessing


In [94]:
Matrix([w0_trial, thx_trial, thy_trial])

Matrix([
[          A_{mn}*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)],
[ pi*A_{mn}*n*sin(pi*m*x/L_x)*cos(pi*n*y/L_y)/L_y],
[-pi*A_{mn}*m*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x]])

In [95]:
kxx = kxx.subs(trial).expand().doit()
kxx = kxx.simplify().expand()
kyy = kyy.subs(trial).expand().doit()
kyy = kyy.simplify().expand()
kxy = kxy.subs(trial).expand().doit()
kxy = kxy.simplify().expand()
Matrix([kxx, kyy, kxy])

Matrix([
[     pi**2*A_{mn}*m**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x**2],
[     pi**2*A_{mn}*n**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y**2],
[-2*pi**2*A_{mn}*m*n*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/(L_x*L_y)]])

In [96]:
mx = mx.subs(trial).expand().doit()
mx = mx.simplify().expand()
my = my.subs(trial).expand().doit()
my = my.simplify().expand()
mxy = mxy.subs(trial).expand().doit()
mxy = mxy.simplify().expand()
vx = vx.subs(trial).expand().doit()
vx = vx.simplify().expand()
vy = vy.subs(trial).expand().doit()
vy = vy.simplify().expand()
Matrix([mx, my, mxy, vx, vy])

Matrix([
[                                                                                                                           pi**2*A_{mn}*D_11*m**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x**2 + pi**2*A_{mn}*D_12*n**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y**2],
[                                                                                                                           pi**2*A_{mn}*D_12*m**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_x**2 + pi**2*A_{mn}*D_22*n**2*sin(pi*m*x/L_x)*sin(pi*n*y/L_y)/L_y**2],
[                                                                                                                                                                                      -2*pi**2*A_{mn}*D_66*m*n*cos(pi*m*x/L_x)*cos(pi*n*y/L_y)/(L_x*L_y)],
[pi**3*A_{mn}*D_11*m**3*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/L_x**3 + pi**3*A_{mn}*D_12*m*n**2*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/(L_x*L_y**2) + 2*pi**3*A_{mn}*D_66*m*n**2*sin(pi*n*y/L_y)*cos(pi*m*x/L_x)/(L_x*L_y**2) + r_{mn}*sin(pi*n*y/L_y)*c